In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.python.client import device_lib

In [2]:
#關閉GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"

print("確認環境：")
print(sys.executable, "\n")

print("確認GPU：")
print(device_lib.list_local_devices())

確認環境：
C:\Users\Mu-Ping\anaconda3\python.exe 

確認GPU：
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16884897762161226699
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5732106240
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14819535145892202252
physical_device_desc: "device: 0, name: GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


In [3]:
# error：Blas xGEMM launch failed
# 原因：https://blog.csdn.net/mch2869253130/article/details/88863779

# 解決辦法：
# 1.
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)
    
# 2.
config = tf.compat.v1.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.compat.v1.Session(config=config)

#tf.compat.v1 為確保高版本的TF支持低版本的TF代碼